# HydroCell Quantum Tunneling Simulation

This notebook demonstrates the quantum mechanical tunneling effects in a hydrogen fuel cell model, visualizing how electrons traverse the proton exchange membrane barrier.

In [ ]:
import sys
sys.path.append("../src")

import numpy as np
import matplotlib.pyplot as plt
from src.hydrocell import FuelCell

# Create fuel cell with parameters optimized for observable tunneling
cell = FuelCell(
    T=298.15, 
    PH2=1.0, 
    PO2=1.0, 
    membrane_resistance=0.05,
    barrier_width=0.5,  # nm - thinner barrier shows more tunneling
    barrier_height=3.0,  # eV
    electron_energy=2.7   # eV - closer to barrier height
)

# Run simulation
result = cell.simulate(load_resistance=0.3)

# Print results
for k, v in result.items():
    print(f"{k.capitalize()}: {v:.6f}")

## Wavefunction Visualization

Below we visualize the electron wavefunction and the potential barrier. In quantum mechanics, electrons behave as waves that can penetrate barriers even when their energy is less than the barrier height - a phenomenon known as quantum tunneling.

The key elements in the visualization:
- **Probability density** ($|\psi|^2$): Represents the probability of finding the electron at each position
- **Potential barrier**: The energy barrier electrons must overcome
- **Electron energy level**: When below the barrier height, classical physics predicts no transmission, but quantum mechanics allows tunneling

In [ ]:
try:
    x, psi = cell.solve_schrodinger()
    
    # Convert to nanometers for display
    x_nm = x * 1e9
    
    # Calculate potential energy profile (in eV for display)
    V_profile = np.array([cell.potential_barrier(pos) / cell.e for pos in x])
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # Plot probability density with proper normalization
    prob_density = np.abs(psi)**2
    if np.max(prob_density) > 0:  # Avoid division by zero
        prob_density = prob_density / np.max(prob_density)  # Normalize for visibility
    
    ax1.plot(x_nm, prob_density, 'b', linewidth=2, label='Probability density $|psi|^2$')
    ax1.fill_between(x_nm, 0, prob_density, alpha=0.3, color='blue')
    ax1.set_ylabel('Normalized probability density', fontsize=12)
    ax1.set_title('Electron Wavefunction', fontsize=14)
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # Plot potential barrier and electron energy level
    ax2.plot(x_nm, V_profile, 'r', linewidth=2, label='Potential barrier')
    ax2.axhline(y=cell.electron_energy, color='g', linestyle='--', 
               label=f'Electron energy ({cell.electron_energy} eV)')
    ax2.fill_between(x_nm, 0, V_profile, where=(V_profile > 0), 
                    alpha=0.3, color='red')
    ax2.set_xlabel('Position (nm)', fontsize=12)
    ax2.set_ylabel('Energy (eV)', fontsize=12)
    ax2.set_title('Potential Energy Profile', fontsize=14)
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print tunneling metrics
    tunneling_prob = cell.compute_tunneling_probability()
    print(f"Tunneling probability: {tunneling_prob:.6f}")
    print(f"Quantum factor: {cell.quantum_enhanced_current(1.0):.6f}")
    
except Exception as e:
    print(f"Error in visualization: {str(e)}")
    # Fallback to simple plot
    plt.figure(figsize=(10, 6))
    plt.title("Potential Barrier")
    barrier_x = np.linspace(-2*cell.barrier_width_m, 3*cell.barrier_width_m, 1000)
    barrier_y = [cell.potential_barrier(x)/cell.e for x in barrier_x]
    plt.plot(barrier_x*1e9, barrier_y)
    plt.axhline(y=cell.electron_energy, color='g', linestyle='--')
    plt.xlabel("Position (nm)")
    plt.ylabel("Energy (eV)")
    plt.show()

## Mathematical Foundation of Tunneling

The tunneling probability through a rectangular barrier is given by:

$$T \approx \exp\left(-2\kappa L\right)$$

where:
- $L$ is the barrier width
- $\kappa = \sqrt{\frac{2m(V_0-E)}{\hbar^2}}$ is the decay constant
- $V_0$ is the barrier height
- $E$ is the electron energy
- $m$ is the electron mass
- $\hbar$ is the reduced Planck constant

This explains why:
1. Thinner barriers (smaller $L$) have higher tunneling probabilities
2. Lower barriers (smaller $V_0$) are easier to tunnel through
3. Higher energy electrons (larger $E$) tunnel more readily

In [ ]:
# Demonstrate the effect of barrier parameters on tunneling probability
import pandas as pd

# Parameter ranges to test
widths = np.linspace(0.2, 1.0, 5)  # nm
heights = np.linspace(2.5, 4.0, 4)  # eV

# Create results table
results = []
for w in widths:
    for h in heights:
        test_cell = FuelCell(
            T=298.15, PH2=1.0, PO2=1.0, membrane_resistance=0.05,
            barrier_width=w, barrier_height=h, electron_energy=2.7
        )
        prob = test_cell.compute_tunneling_probability()
        results.append({'Width (nm)': w, 'Height (eV)': h, 'Tunneling Probability': prob})

# Display as DataFrame
df = pd.DataFrame(results)
pd.pivot_table(df, values='Tunneling Probability', 
               index='Width (nm)', columns='Height (eV)', 
               aggfunc='mean').style.format("{:.8f}")